# Reference

https://www.youtube.com/watch?v=mTsZL-VmRVE

# Caution

Callback 할 때에는 jupyter file 새로고침 하고, 다시 돌려야함
- 그렇지 않으면, 웹에 결과가 제대로 표기 안되는 경우가 자주 발생

# Library

In [1]:
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import plotly.express as px
import plotly.graph_objs as go

import os
import pandas as pd
import numpy as np

# Data

In [2]:
# read data
data_path = os.path.dirname(os.getcwd()) + "/data" + "/Mutual-Funds.csv"
df = pd.read_csv(data_path)

# print
df.head(3)

,fund_name,fund_extended_name,category,fund_family,net_assets,ytd_return,fund_yield,morningstar_rating,inception_date,investment,...,consumer_cyclical,financial_services,real_estate,consumer_defensive,healthcare,utilities,communication_services,energy,industrials,technology
0,FAAAX,Franklin K2 Alternative Strategies Fund Class A,Multialternative,Franklin Templeton Investments,1.210000e+09,4.35,0.00,4,10/10/2013,Growth,...,16.19,8.32,0.25,1.87,21.54,0.73,1.51,1.17,19.11,22.27
1,FAACX,Nuveen All-American Municipal Bond Fund Class C2,Muni National Long,Nuveen,4.060000e+09,3.22,2.96,4,6/1/1993,Growth,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,FAAFX,Fairholme Allocation Fund,Allocation - 30% to 50% Equity,Fairholme,7.662000e+07,11.21,1.02,1,12/30/2010,Growth,...,15.97,0.00,60.35,0.00,0.00,0.00,0.00,0.00,0.00,0.00


# Dash - app

In [3]:
app = dash.Dash()

## > Date range picker

In [4]:
colors = ["black", "blue", "red", "yellow", "pink", "orange"]

In [5]:
app.layout = html.Div(
    children=[
        dcc.Dropdown(
            id='my-dropdown',
            multi=True,
            options=[{'label': x, 'value': x} for x in sorted(df.fund_extended_name.unique())],
            value=[
                "Fidelity 500 Index Fund", "Fidelity Advisor Freedom 2035 Fund Class A",
                "Fidelity Freedom 2035 Fund"
            ]
        ),
        html.Button(
            id='my-button',
            n_clicks=0,
            children="Show breakdown"
        ),
        dcc.Graph(
            id='graph-output',
            figure={}
        ),

        html.Div(
            id="sentence-output",
            children=[
                "This is the color I love"
            ],
            style={}
        ),
        dcc.RadioItems(
            id='my-radioitem',
            value="black",
            options=[
                {'label': c, 'value': c} for c in colors
            ]
        ),
    ]
)

In [6]:
# State 의 경우, Input 이 trigger 되어야만 ㅡ 결과가 바뀜
# 즉, 목록에서 항목을 선택한 후 ㅡ 버튼을 클릭(Input이 trigger) 해야 됨
@app.callback(
    Output(component_id='graph-output', component_property='figure'),
    [Input(component_id='my-button', component_property='n_clicks')],
    [State(component_id='my-dropdown', component_property='value')],
    prevent_initial_call=False
)
def update_my_graph(
    n,
    val_chosen
):
    if len(val_chosen) > 0:
        print(n)
        print(f"value user chose: {val_chosen}")
        print(type(val_chosen))
        dff = df[df["fund_extended_name"].isin(val_chosen)]
        fig = px.pie(dff, values="ytd_return", names="fund_extended_name", title="Year-to-Date Returns")
        fig.update_traces(textinfo="value+percent").update_layout(title_x=0.5)
        return fig
    elif len(val_chosen) == 0:
        raise dash.exceptions.PreventUpdate

In [7]:
if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Feb/2021 14:09:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2021 14:09:48] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2021 14:09:48] "GET /_dash-layout HTTP/1.1" 200 -


0
value user chose: ['Fidelity 500 Index Fund', 'Fidelity Advisor Freedom 2035 Fund Class A', 'Fidelity Freedom 2035 Fund']
<class 'list'>


127.0.0.1 - - [26/Feb/2021 14:09:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2021 14:09:53] "POST /_dash-update-component HTTP/1.1" 200 -


1
value user chose: ['Fidelity 500 Index Fund', 'Fidelity Advisor Freedom 2035 Fund Class A', 'Fidelity Freedom 2035 Fund', 'AB FlexFee US Thematic Portfolio Advisor Class']
<class 'list'>
